In [1]:
# qu.ApiConfig.api_key = "wFHCRKsuVAJ8_oiWPDzA"

In [2]:
import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path
class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod

class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

In [3]:
sys.meta_path.append(NotebookFinder())

from Utils import is_business_day, is_holiday, expiration_date

importing Jupyter notebook from Utils.ipynb


In [4]:
import quandl, numpy, math, datetime
from dateutil.relativedelta import relativedelta
from utils import expiration_date, is_business_day, is_holiday
quandl.ApiConfig.api_key = "tjxFewEnckmn8zjjsHSZ"

"""
Updated Friday November 23, 2018 at ~4PM
@author: Phil
"""

def trade_signal(signal_date,commodity,months_ahead):

    #Confirms signal_date is not a business day or holiday
    if not is_business_day(signal_date) or (is_holiday(signal_date)):
        return -2

    #Finds the near and far commodity ticker
    #Crude oil
    if commodity == "WTI":
        #WTI crude
        near_commodity = "CHRIS/CME_CL1"
        far_commodity = ''.join(["CHRIS/CME_CL",str(months_ahead+1)])
    elif commodity == "BRENT":
        #Brent crude
        near_commodity = "CHRIS/ICE_B1"
        far_commodity = ''.join(["CHRIS/ICE_B",str(months_ahead+1)])
    #Natural gas
    elif commodity == "NG":
        #US natural gas
        near_commodity = "CHRIS/CME_NG1"
        far_commodity = ''.join(["CHRIS/CME_NG",str(months_ahead+1)])
    elif commodity == "UKNG":
        #UK natural gas
        near_commodity = "CHRIS/ICE_M1"
        far_commodity = ''.join(["CHRIS/ICE_M",str(months_ahead+1)])
    #Cattle
    elif commodity == "LC":
        #Live cattle
        near_commodity = "CHRIS/CME_LC1"
        far_commodity = ''.join(["CHRIS/CME_LC",str(months_ahead+1)])
    elif commodity == "FC":
        #Feeder cattle
        near_commodity = "CHRIS/CME_FC1"
        far_commodity = ''.join(["CHRIS/CME_FC",str(months_ahead+1)])
    #Distillates
    elif commodity == "RBOBGAS":
        #RBOB Gas
        near_commodity = "CHRIS/CME_RB1"
        far_commodity = ''.join(["CHRIS/CME_RB",str(months_ahead+1)])
    elif commodity == "GASOIL":
        #Gas oil
        near_commodity = "CHRIS/ICE_G1"
        far_commodity = ''.join(["CHRIS/ICE_G",str(months_ahead+1)])
    elif commodity == "HEATOIL":
        #Heating oil
        #Only goes out to the fourth month (fifth future)
        near_commodity = "CHRIS/ICE_O1"
        far_commodity = ''.join(["CHRIS/ICE_O",str(months_ahead+1)])
    #Wheat
    elif commodity == "WHEAT":
        #Plain ol' wheat
        near_commodity = "CHRIS/CME_W1"
        far_commodity = ''.join(["CHRIS/CME_W",str(months_ahead+1)])
    elif commodity == "KWHEAT":
        #Kansas City hard red wheat
        #Only goes out to the fifth month (sixth future)
        near_commodity = "CHRIS/CME_KW1"
        far_commodity = ''.join(["CHRIS/CME_KW",str(months_ahead+1)])
    elif commodity == "MWHEAT":
        #Milling wheat
        #Only goes out to the fifth month (sixth future)
        near_commodity = "CHRIS/LIFFE_EBM1"
        far_commodity = ''.join(["CHRIS/LIFFE_EBM",str(months_ahead+1)])
    #Catch all for commodities not on the list
    else:
        return -3

    #Converts the date to the right format and fetches the near and far future
    if signal_date.month <10:
        signal_date_month_string = ''.join(["0",str(signal_date.month)])
    else:
        signal_date_month_string = str(signal_date.month)
        
    if signal_date.day < 10:
        signal_date_day_string = ''.join(["0",str(signal_date.day)])
    else:
        signal_date_day_string = str(signal_date.day)
        
    signal_date_string = ''.join(['"',str(signal_date.year),'-',signal_date_month_string,'-',signal_date_day_string,'"'])
##    print(signal_date_string)

    try:
        near = quandl.get(near_commodity,start_date=signal_date_string,end_date=signal_date_string,returns="numpy")
        for item in near:
            near_price = float(item[4])
##            print(near_price)
    except:
        return -4

    try:
        far = quandl.get(far_commodity,start_date=signal_date_string,end_date=signal_date_string,returns="numpy")
        for item in far:
            far_price = float(item[4])
##        print(far_price)
    except:
        return -5

    #Figures out the dates of the the near and far futures, days between the two dates
    near_date = expiration_date(signal_date.year,signal_date.month)
    far_date = near_date + relativedelta(months=+months_ahead)
    far_date = expiration_date(far_date.year,far_date.month)
##    print(near_date)
##    print(far_date)
    if near_date.date() <= signal_date:
##        print(True)
        near_date = near_date + relativedelta(months=+1)
        far_date = far_date + relativedelta(months=+1)
##        print(near_date)
##        print(far_date)
    delta = far_date - near_date
##    print(delta.days)

    #Calculates the signal and returns it
    try:
        signal = (math.log(near_price) - math.log(far_price)) * 365 / delta.days
    except:
        return -6
    return signal

importing Jupyter notebook from utils.ipynb


In [5]:
from datetime import timedelta, date
import pandas as pd
import numpy as np
import quandl, math
from dateutil.relativedelta import relativedelta
from utils import expiration_date, is_business_day, is_holiday

quandl.ApiConfig.api_key = "tjxFewEnckmn8zjjsHSZ"

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)
        
start_date = date(2015, 1, 1)
end_date = date(2015, 1, 20)
signal = []
spot_priceWTI = []
future_priceWTI = []
spot_priceBRENT = []
future_priceBRENT = []


spotWTI = quandl.get("EIA/PET_RWTC_D",start_date=start_date,end_date=end_date,returns="numpy")
spotBRENT = quandl.get("EIA/PET_RBRTE_D",start_date=start_date,end_date=end_date,returns="numpy")

futureWTI = quandl.get("CHRIS/CME_CL1",start_date=start_date,end_date=end_date,returns="numpy")
futureBRENT = quandl.get("CHRIS/ICE_B1",start_date=start_date,end_date=end_date,returns="numpy")

for item in spotWTI:
    s_price = float(item[1])
    spot_priceWTI.extend([s_price])
    
for item in spotBRENT:
    s_price = float(item[1])
    spot_priceBRENT.extend([s_price])
    
for item in futureWTI: 
    f_price = float(item[4])
    future_priceWTI.extend([f_price])  
    
for item in futureBRENT: 
    f_price = float(item[4])
    future_priceBRENT.extend([f_price])

    
for single_date in daterange(start_date, end_date):
    sig = trade_signal(single_date,"WTI",6)
    signal.extend([sig])
    

In [6]:
retrn = np.zeros(len(spot_priceWTI),dtype=np.float)
retrn[0] = 0
returnWTI = np.zeros(len(spot_priceWTI),dtype=np.float)
returnBrent = np.zeros(len(spot_priceWTI),dtype=np.float)
pnl = np.zeros(len(spot_priceWTI),dtype=np.float)
pnl[0] = 0
Init_capital = 1000
capital = np.zeros(len(signal),dtype=np.float)
capital[0]=Init_capital


for i in range(1,len(spot_priceWTI)-1):
    if abs(signal[i])>.01 and abs(signal[i])<2:
        returnWTI[i] = np.sign(signal[i])*(((spot_priceWTI[i]-future_priceWTI[i])/future_priceWTI[i])+((spot_priceWTI[i]-spot_priceWTI[i-1])/spot_priceWTI[i-1]))
        returnBrent[i] = np.sign(signal[i])*(((spot_priceBRENT[i]-future_priceBRENT[i])/future_priceBRENT[i])+((spot_priceBRENT[i]-spot_priceBRENT[i-1])/spot_priceBRENT[i-1]))
        retrn[i] = returnWTI[i]+returnBrent[i]
        capital[i]=(1+retrn[i])*capital[i-1]
        pnl[i]=capital[i]-capital[i-1]
        total_PNL=np.cumsum(pnl)
    else:
        retrn[i]=0
        pnl[i]=0
        total_PNL=np.cumsum(pnl)
        capital[i]=capital[i-1]

In [7]:
print(retrn)
print(capital)
print(pnl)
print(total_PNL)

[ 0.          0.07596839  0.          0.          0.00973353  0.07414677
 -0.01356107  0.11273554  0.09949997  0.          0.         -0.02498906
  0.        ]
[1000.         1075.96838596 1075.96838596 1075.96838596 1086.44135675
 1166.9974693  1151.17173778 1280.94970111 1408.40415964 1408.40415964
 1408.40415964 1373.20946076    0.            0.            0.
    0.            0.            0.            0.        ]
[  0.          75.96838596   0.           0.          10.47297079
  80.55611255 -15.82573153 129.77796333 127.45445853   0.
   0.         -35.19469888   0.        ]
[  0.          75.96838596  75.96838596  75.96838596  86.44135675
 166.9974693  151.17173778 280.94970111 408.40415964 408.40415964
 408.40415964 373.20946076 373.20946076]


Questions

1). Electricity cannot really be stored, gives unrealistic results avoid this
2). Not picky about financial costs
    Bid/Ask Spread - necessary, Brokerage Fees, etc.
3). Money management/margins extra not especially important
4). Can combine signals (extension)
5). How many products/pairs? - Even
    Long and short for each commodity, correlated does not matter so much
    Want long term and short term one (1 year too long)
    Across as wide of a universe as possible
    Depends on universe, but want minimum 4 pairs (more the better)
6). Holding period - 1 month?
7). Have to decide when to roll? Recalculate signals when rolling new contract?